In [2]:
# !pip install -qU wtpsplit pandas tqdm datasets

In [ ]:
from wtpsplit import SaT
import pandas as pd
import re
from tqdm import tqdm
from datasets import load_dataset

from utils import get_device

https://huggingface.co/segment-any-text/sat-12l-sm

In [ ]:
device = get_device()
sat = SaT('sat-12l')
sat.half().to(device)
sat

https://huggingface.co/datasets/ubaada/booksum-complete-cleaned

In [ ]:
book_data = load_dataset('ubaada/booksum-complete-cleaned', 'books')

In [ ]:
print(book_data['test'][0]['text'][:500])

PART I Book I. The History Of A Family Chapter I. Fyodor Pavlovitch Karamazov


Alexey Fyodorovitch Karamazov was the third son of Fyodor Pavlovitch
Karamazov, a land owner well known in our district in his own day, and
still remembered among us owing to his gloomy and tragic death, which
happened thirteen years ago, and which I shall describe in its proper
place. For the present I will only say that this "landowner"--for so we
used to call him, although he hardly spent a day of his life on his 


In [ ]:
post_splits = []

for book in tqdm(book_data['test'], desc='Processing chapters'):
  split = sat.split(book['text'])
  for i, segment in enumerate(split):
    post_splits.append({'index': i, 'bid': book['bid'], 'segment': segment})

Processing chapters: 100%|██████████| 17/17 [12:16<00:00, 43.34s/it]


In [6]:
post_splits[0]

{'index': 0, 'bid': '28054', 'segment': 'PART I '}

In [7]:
df = pd.DataFrame(post_splits)

In [8]:
df.head(5)

,index,bid,segment
0,0,28054,PART I
1,1,28054,Book I. The History Of A Family Chapter I.
2,2,28054,Fyodor Pavlovitch Karamazov\n\n\n
3,3,28054,Alexey Fyodorovitch Karamazov was the third so...
4,4,28054,"For the present I will only say that this ""lan..."


In [9]:
df['segment'] = df['segment'].apply(lambda x: re.sub(r'(?<!\n)\n(?!\n)', ' ', x))
df['segment'] = df['segment'].apply(lambda x: re.sub(r'\n{2,}', '', x)) 
df['segment'] = df['segment'].apply(lambda x: re.sub(r'\n', ' ', x))
df['segment'] = df['segment'].apply(lambda x: x.rstrip())

In [10]:
df.to_csv('processed_chapters.csv', index=False)

In [11]:
df.loc[df['bid'] == '28054']

,index,bid,segment
0,0,28054,PART I
1,1,28054,Book I. The History Of A Family Chapter I.
2,2,28054,Fyodor Pavlovitch Karamazov
3,3,28054,Alexey Fyodorovitch Karamazov was the third so...
4,4,28054,"For the present I will only say that this ""lan..."
...,...,...,...
18173,18173,28054,If the deceased was a priest as well as a monk...
18174,18174,28054,7 i.e. a chime of bells.
18175,18175,28054,8 Literally:
18176,18176,28054,"""Did you get off with a long nose made at you?..."
